In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install pycaret

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 320 kB 30.2 MB/s 
     |████████████████████████████████| 167 kB 68.8 MB/s 
     |████████████████████████████████| 122 kB 54.6 MB/s 
     |████████████████████████████████| 6.8 MB 11.7 MB/s 
     |████████████████████████████████| 2.0 MB 37.3 MB/s 
     |████████████████████████████████| 88 kB 4.9 MB/s 
     |████████████████████████████████| 1.3 MB 27.9 MB/s 
     |████████████████████████████████| 262 kB 67.3 MB/s 
     |████████████████████████████████| 10.4 MB 54.1 MB/s 
     |████████████████████████████████| 17.9 MB 580 kB/s 
     |████████████████████████████████| 1.7 MB 52.9 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 56 kB 5.2 MB/s 
     |████████████████████████████████|

In [2]:
!pip install markupsafe==2.0.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 2.1.1
    Uninstalling MarkupSafe-2.1.1:
      Successfully uninstalled MarkupSafe-2.1.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas-profiling 3.2.0 requires markupsafe~=2.1.1, but you have markupsafe 2.0.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [3]:
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
import seaborn as sns
import platform
import datetime
from datetime import datetime
# 그래프 폰트 설정
# Window
if platform.system() == 'Windows':
    plt.rc('font', family='Malgun Gothic')
elif platform.system() == 'Darwin': # Mac
    plt.rc('font', family='AppleGothic')
else: #linux
    plt.rc('font', family='NanumGothic')

# 그래프에 마이너스 표시가 되도록 변경
plt.rcParams['axes.unicode_minus'] = False

# 파이썬 ≥3.5 필수
import sys
assert sys.version_info >= (3, 5)

# 사이킷런 ≥0.20 필수
# 여러가지 머신러닝 모델을 가지고 있는 라이브러리 *가장 많이 사용* 
import sklearn
# 공통 모듈 임포트
import numpy as np
import os
import pandas as pd
from sklearn.model_selection import train_test_split 

# 깔끔한 그래프 출력을 위해
plt.rcParams['font.family'] = 'NanumGothic'
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

In [4]:
cd '/content/drive/MyDrive/2022/공모전'

/content/drive/MyDrive/2022/공모전


### 데이터 전처리

In [51]:
#data불러오기
export = pd.read_csv('export.csv')
submission = pd.read_csv('submission.csv', encoding= 'unicode_escape')
singapore = pd.read_csv('singapore.csv', encoding= 'unicode_escape')

In [52]:
# 변수명 변경
export.columns = ['기간','한국_수출금액', '한국의_중국_수출', '한국의_미국_수출', '한국의_베트남_수출','한국의_홍콩_수출','한국의_일본_수출']
singapore.columns = ['기간','한국의_싱가포르_수출']
submission.columns = ['기간','한국_수출금액']

In [53]:
# 기간 날짜데이터로 바꿔주기
singapore['기간'] = singapore['기간'].astype(str).str.replace('.', '-')
export['기간'] = pd.to_datetime(export['기간'])
singapore['기간'] = pd.to_datetime(singapore['기간'])
submission['기간'] = pd.to_datetime(submission['기간'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  


In [54]:
singapore['한국의_싱가포르_수출'] = singapore['한국의_싱가포르_수출']*1000

In [55]:
# train 데이터와 singapore 데이터를 합쳐준다.
df = pd.merge(export,singapore)

In [28]:
# df['한국의_싱가포르_수출'] = df['한국의_싱가포르_수출'].str.replace(",", "")
# df['한국의_싱가포르_수출'] = df['한국의_싱가포르_수출'].str.replace(" ", "")
# df["한국의_싱가포르_수출"] = pd.to_numeric(df["한국의_싱가포르_수출"])

In [56]:
df

,기간,한국_수출금액,한국의_중국_수출,한국의_미국_수출,한국의_베트남_수출,한국의_홍콩_수출,한국의_일본_수출,한국의_싱가포르_수출
0,2000-01-01,12162424632,1235215271,2609787984,109742698,782348732,1551722070,464550000
1,2000-02-01,12676042304,1273117309,2554309883,112395157,823990122,1518844843,379588000
2,2000-03-01,14429703086,1437377223,2946820669,150134562,911648256,1783564618,611323000
3,2000-04-01,13522040017,1366876604,2698029849,169146208,863243219,1582707525,430097000
4,2000-05-01,14636754971,1663240667,3167181403,169411680,887081090,1671039709,551510000
...,...,...,...,...,...,...,...,...
259,2021-08-01,53165103786,13881327880,7704223847,4850337611,3415828590,2522980082,1116604000
260,2021-09-01,55913862375,14291980345,7983934107,5101701490,3852088155,2541017857,1332202000
261,2021-10-01,55660372637,14415867215,8066782602,4904971459,2739662629,2614683864,1353718000
262,2021-11-01,60330542263,15272549044,8133640339,5478296945,3232583381,2813250497,1463579000


In [57]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 264 entries, 0 to 263
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   기간           264 non-null    datetime64[ns]
 1   한국_수출금액      264 non-null    int64         
 2   한국의_중국_수출    264 non-null    int64         
 3   한국의_미국_수출    264 non-null    int64         
 4   한국의_베트남_수출   264 non-null    int64         
 5   한국의_홍콩_수출    264 non-null    int64         
 6   한국의_일본_수출    264 non-null    int64         
 7   한국의_싱가포르_수출  264 non-null    int64         
dtypes: datetime64[ns](1), int64(7)
memory usage: 18.6 KB


In [62]:
submission

,기간,한국_수출금액
0,2022-01-01,NaN
1,2022-02-01,NaN
2,2022-03-01,NaN
3,2022-04-01,NaN
4,2022-05-01,NaN
5,2022-06-01,NaN


### pycaret

In [73]:
#함수 호출
from pycaret.regression import * 
# 설정
reg = setup(data = df, target = '한국_수출금액', train_size = 0.8,  use_gpu = True, data_split_shuffle=True, normalize = True, session_id=42,silent= True)

,Description,Value
0,session_id,42
1,Target,한국_수출금액
2,Original Data,"(264, 8)"
3,Missing Values,False
4,Numeric Features,6
5,Categorical Features,0
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(211, 27)"


In [74]:
# RMSE 기준으로 가장 좋은 모델 5가지를 선택해준다.
best_3 = compare_models(sort = 'RMSE', n_select = 3)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,1.081241e+09,2.316359e+18,1.494632e+09,0.9860,0.0403,0.0312,0.737
ridge,Ridge Regression,1.164708e+09,2.516143e+18,1.557617e+09,0.9850,0.0466,0.0359,0.012
llar,Lasso Least Angle Regression,1.150654e+09,2.534062e+18,1.559418e+09,0.9849,0.0454,0.0347,0.018
lr,Linear Regression,1.150656e+09,2.534069e+18,1.559420e+09,0.9849,0.0454,0.0347,0.011
lasso,Lasso Regression,1.150657e+09,2.534074e+18,1.559422e+09,0.9849,0.0454,0.0347,0.014
lar,Least Angle Regression,1.150653e+09,2.534604e+18,1.559557e+09,0.9849,0.0454,0.0347,0.017
gbr,Gradient Boosting Regressor,1.203058e+09,2.718432e+18,1.624409e+09,0.9835,0.0449,0.0354,0.074
rf,Random Forest Regressor,1.235066e+09,2.880518e+18,1.678071e+09,0.9822,0.0454,0.0358,0.771
lightgbm,Light Gradient Boosting Machine,1.300139e+09,3.044280e+18,1.718410e+09,0.9812,0.0494,0.0390,0.028
omp,Orthogonal Matching Pursuit,1.384330e+09,3.780180e+18,1.920430e+09,0.9772,0.0508,0.0387,0.010


In [75]:
blended_3 = blend_models(estimator_list= best_3, fold=12, optimize='RMSE')
pred_holdout = predict_model(blended_3)
final_model = finalize_model(blended_3)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,9.656111e+08,1.977075e+18,1.406085e+09,0.9863,0.0380,0.0281
1,1.111206e+09,1.932706e+18,1.390218e+09,0.9847,0.0379,0.0302
2,1.105228e+09,1.950115e+18,1.396465e+09,0.9846,0.0390,0.0311
3,4.362034e+08,3.514248e+17,5.928110e+08,0.9984,0.0231,0.0175
4,1.346259e+09,4.015380e+18,2.003841e+09,0.9810,0.0527,0.0379
5,9.871443e+08,1.873385e+18,1.368717e+09,0.9899,0.0407,0.0311
6,9.025271e+08,1.093794e+18,1.045846e+09,0.9946,0.0346,0.0299
7,1.010207e+09,2.062665e+18,1.436198e+09,0.9887,0.0405,0.0316
8,9.533852e+08,1.525048e+18,1.234928e+09,0.9909,0.0361,0.0276


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Voting Regressor,1.099038e+09,2.360992e+18,1.536552e+09,0.9871,0.0428,0.0345


In [76]:
data = {
    '기간' : ['2022-01-01','2022-02-01','2022-03-01','2022-04-01','2022-05-01','2022-06-01'],
    '한국_수출금액' : [np.nan,np.nan,np.nan,np.nan,np.nan,np.nan],
    '한국의_중국_수출' : [np.nan,np.nan,np.nan,np.nan,np.nan,np.nan],
    '한국의_미국_수출' : [np.nan,np.nan,np.nan,np.nan,np.nan,np.nan],
    '한국의_베트남_수출' : [np.nan,np.nan,np.nan,np.nan,np.nan,np.nan],
    '한국의_홍콩_수출' : [np.nan,np.nan,np.nan,np.nan,np.nan,np.nan],
    '한국의_일본_수출' : [np.nan,np.nan,np.nan,np.nan,np.nan,np.nan],
    '한국의_싱가포르_수출' : [np.nan,np.nan,np.nan,np.nan,np.nan,np.nan]
}
test = pd.DataFrame(data)
test['기간'] = pd.to_datetime(test['기간'])

In [77]:
pred = predict_model(final_model, test) 

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Voting Regressor,0,0,0,0,0,0


In [78]:
pred

,기간,한국_수출금액,한국의_중국_수출,한국의_미국_수출,한국의_베트남_수출,한국의_홍콩_수출,한국의_일본_수출,한국의_싱가포르_수출,Label
0,2022-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.599261e+10
1,2022-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.630955e+10
2,2022-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.639192e+10
3,2022-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.625845e+10
4,2022-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.597744e+10
5,2022-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.688408e+10


In [79]:
pred.rename(columns = {"Label": "target"}, inplace = True)

In [80]:
submission['한국_수출금액'] = pred['target']

In [81]:
submission.to_csv('pycaret.csv', index = False)

### ARIMA